<a href="https://colab.research.google.com/github/BaseKan/aiday_training_resources/blob/main/TF_basics/Tensorflow_training_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Data downloaden
!curl -L -c cookies.txt 'https://docs.google.com/uc?export=download&id=14P4Pv7Lq-pvYTjTRKld_ffrvFk6C7ntv' | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt    
!curl -L -b cookies.txt -o 'telecom_churn.csv' 'https://docs.google.com/uc?export=download&id=14P4Pv7Lq-pvYTjTRKld_ffrvFk6C7ntv&confirm='$(<confirm.txt)
!rm -f confirm.txt cookies.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0    307      0 --:--:--  0:00:01 --:--:--   307
100  631k  100  631k    0     0   341k      0  0:00:01  0:00:01 --:--:--  341k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0    293      0 --:--:--  0:00:01 --:--:--   293
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100  631k  100  631k    0     0   304k      0  0:00:02  0:00:02 --:--:--  304k
